<a href="https://colab.research.google.com/github/lordjuacs/tesis/blob/master/tesis_embeddings_falcon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers einops accelerate langchain bitsandbytes sentence_transformers faiss-gpu


In [ ]:
!nvidia-smi

Fri Dec  1 23:05:15 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Import necessary libraries
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer, pipeline
from langchain import HuggingFacePipeline, PromptTemplate, LLMChain
import torch
import faiss


In [ ]:
# Load your dataset
dataset_path = "tesis_dataset.csv"
df = pd.read_csv(dataset_path)


In [ ]:
# Load a sentence embedding model (example: 'paraphrase-MiniLM-L6-v2')
embedding_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Create embeddings for the dataset
dataset_embeddings = embedding_model.encode(df['pregunta'].tolist())


In [ ]:
# Store the embeddings in a vector database (use your preferred database library)

# For example, using faiss
index = faiss.IndexFlatL2(dataset_embeddings.shape[1])
index.add(dataset_embeddings)


In [ ]:
# User inserts custom question
user_question = "Explícame qué es el intercambio estudiantil"

# Convert user question to embedding
user_question_embedding = embedding_model.encode(user_question)


In [ ]:
# Send to LLM Chain for answer generation
model = "vilsonrodrigues/falcon-7b-instruct-sharded"

tokenizer = AutoTokenizer.from_pretrained(model)

pipeline = pipeline(
    "text-generation",  # task
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)



In [ ]:
llm = HuggingFacePipeline(pipeline=pipeline, model_kwargs={'temperature': 0.7, 'top_p': 0.7, "generation_config.max_new_tokens": 200, "num_return_sequence": 1})


In [ ]:
# Create the LLM Chain
template = """
Eres un chatabot inteligente que sirve como trabajador en la Universidad de Ingeniería y Tecnología (UTEC). Ayúdame a responder la siguiente pregunta de manera directa y concisa.
Pregunta: {pregunta}
Respuesta:"""
prompt = PromptTemplate(template=template, input_variables=["pregunta"])

llm_chain = LLMChain(prompt=prompt, llm=llm)


In [ ]:
# Edit this cell to test different questions and run to get model answers

# Example question
user_question = "¿Qué becas hay?"

# Convert user question to embedding
user_question_embedding = embedding_model.encode(user_question)

# Apply similarity search
k = 5  # Number of neighbors to retrieve
_, most_similar_indices = index.search(user_question_embedding.reshape(1, -1), k)

# Retrieve the most similar question-answer pair
most_similar_index = most_similar_indices[0][0]
most_similar_question = df['pregunta'][most_similar_index]
most_similar_answer = df['respuesta'][most_similar_index]

# Use the retrieved question as input to LLM Chain
generated_answer = llm_chain.run(most_similar_question)

# Display results
print(f"User Question: {user_question}")
print(f"Most Similar Question: {most_similar_question}")
print(f"Retrieved Answer: {most_similar_answer}")
print(f"Generated Answer: {generated_answer}")


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


User Question: ¿Qué becas hay?
Most Similar Question: ¿Qué son las becas financiadas por terceros y cómo se gestionan?
Retrieved Answer: Las becas financiadas por terceros provienen de convenios o auspicios con empresas o entidades y se gestionan estableciendo criterios específicos de selección y otorgamiento.
Generated Answer:  Las becas son programas de apoyo financiero ofrecidos por terceros a estudiantes de tercer ciclo, en particular para que aborden temas específicos a los que están interesados. Se administra en función de la legislación de la administración federal del estado. A su administración se remite un presupuesto y un plan de estudios a cumplir.
A continuación, se presentan algunos puntos clave para administrar las becas financiadas por terceros.
- Aplicar en la Universidad. Una vez aprobada la
